# YOLO11n-SEG Training

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
!nvcc --version

import os
import torch
from IPython import get_ipython


BASE_PATH = os.path.dirname(os.getcwd())
VISIBLE_GPUS = [3,4,6]

os.environ["CUDA_VISIBLE_DEVICES"] = str(VISIBLE_GPUS).replace("]", "").replace("[","").replace(" ", "")
os.environ["NVIDIA_VISIBLE_DEVICES"] = str(VISIBLE_GPUS).replace("]", "").replace("[","").replace(" ", "")
print("CUDA GPUS NUMBER: ", torch.cuda.device_count())

In [ ]:
YAML_RELATIVE_PATH = "dataset/data.yaml"
DATA_YAML = os.path.join(BASE_PATH, YAML_RELATIVE_PATH)


print(BASE_PATH)
print(DATA_YAML)
print(YAML_RELATIVE_PATH)

In [4]:
import numpy as np
import json
import os
import datetime
import pytz
from ultralytics import YOLO

class NPEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)

In [5]:
import numpy as np
import json


class NPEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)

In [ ]:
BATCH_SIZE = 210*3
TRAIN_EPOCH = 100
IM_SIZE = 800
RESULTS_RELATIVE_PATH = 'arquivos/Citologia/RESULTS/Unified/yolo11n/seg'
current_run = datetime.datetime.now(pytz.timezone("America/Sao_Paulo"))
dirname = "{dia}-{mes}-{ano}_{hora}_{minuto}_{segundo}".format(
            dia=current_run.day,
            mes=current_run.month,
            ano=current_run.year, 
            hora=current_run.hour,
            minuto=current_run.minute,
            segundo=current_run.second,
)
PROJECT_PATH = os.path.join(BASE_PATH, RESULTS_RELATIVE_PATH, dirname)
os.makedirs(PROJECT_PATH, exist_ok=True)
model = YOLO('yolo11n-seg.pt')

model.train(data=DATA_YAML, epochs=TRAIN_EPOCH, imgsz=IM_SIZE, batch=BATCH_SIZE, device=VISIBLE_GPUS, project=PROJECT_PATH, exist_ok=True)

In [ ]:
model = YOLO('/model_weights/YOLO11n-seg/best.pt')
PROJECT_PATH = '/path/to/yolo/project'

In [7]:
metrics_test = model.val(save_json=True, project=PROJECT_PATH, name="test", split="test")
metrics_val = model.val(save_json=True, project=PROJECT_PATH, name="val", split="val")


summary_json = {
    "metrics": metrics_val.results_dict,
    "speed": metrics_val.speed,
}
summary_json["metrics"].update({"map50": metrics_val.seg.map50})
summary_json["metrics"].update({"map75": metrics_val.seg.map75})
summary_json["metrics"].update({"f1": metrics_val.seg.f1})

with open(os.path.join(PROJECT_PATH, "val/val_summary.json"), "w") as of:
    json.dump(summary_json, of, cls=NPEncoder)

summary_json = {
    "metrics": metrics_test.results_dict,
    "speed": metrics_test.speed,
}
summary_json["metrics"].update({"map50": metrics_test.seg.map50})
summary_json["metrics"].update({"map75": metrics_test.seg.map75})
summary_json["metrics"].update({"f1": metrics_test.seg.f1})

with open(os.path.join(PROJECT_PATH, "test/test_summary.json"), "w") as of:
    json.dump(summary_json, of, cls=NPEncoder)

Ultralytics 8.3.89 🚀 Python-3.11.10 torch-2.6.0+cu124 CUDA:0 (NVIDIA H100 80GB HBM3, 80995MiB)
YOLO11n-seg summary (fused): 113 layers, 2,838,858 parameters, 0 gradients, 10.2 GFLOPs


val: Scanning /home/jovyan/arquivos/Citologia/Datasets/Unificado/images_rescaled/yolo_format/test/labels.cache... 2472 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2472/2472 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/jovyan/arquivos/Citologia/Datasets/Unificado/images_rescaled/yolo_format/test/images/2019_07_10__14_48__0045_b0s0c0x139843-1600y73546-1200m10229.tiff: 1 duplicate labels removed
val: WARNING ⚠️ /home/jovyan/arquivos/Citologia/Datasets/Unificado/images_rescaled/yolo_format/test/images/2019_07_10__14_48__0045_b0s0c0x144234-1600y21755-1200m3080.tiff: 1 duplicate labels removed
val: WARNING ⚠️ /home/jovyan/arquivos/Citologia/Datasets/Unificado/images_rescaled/yolo_format/test/images/2019_07_10__14_48__0045_b0s0c0x144249-1600y14208-1200m2037.tiff: 1 duplicate labels removed
val: WARNING ⚠️ /home/jovyan/arquivos/Citologia/Datasets/Unificado/images_rescaled/yolo_format/test/images/2019_07_10__14_48__0045_b0s0c0x147120-1600y20684-1200m2933.tiff: 1 duplicate labels removed
val: WARNING ⚠️ /home/jovyan/arquivos/Citologia/Datasets/Unificado/images_rescaled/yolo_format/test/images/2019_07_10__14_48__0045_b0s0c0x155707-1600y56314-1200m7856.tiff: 1 duplicate labels removed
val:


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 155/155 [10:52<00:00,  4.21s/it]


                   all       2472      16240      0.561      0.526      0.509      0.389      0.548      0.523        0.5       0.34
           pap_anormal        629       1688      0.481      0.794      0.643      0.496       0.48      0.793      0.641      0.455
          pap_saudavel         91        249      0.504      0.787      0.696      0.612      0.504      0.787      0.696      0.577
      pap_fora_de_foco        580       1846      0.415      0.367      0.391      0.266      0.415      0.367      0.391      0.242
         pap_sanguinea        301        671      0.423      0.607       0.42      0.316      0.422      0.605      0.419      0.279
        pap_em_divisao          5          5          1          0     0.0862     0.0745          1          0     0.0862     0.0689
      pap_nao_definida          5          9          1          0    0.00351    0.00256          1          0    0.00351    0.00218
         pap_reacional         48        114      0.406      0.138   

val: Scanning /home/jovyan/arquivos/Citologia/Datasets/Unificado/images_rescaled/yolo_format/val/labels.cache... 2448 images, 0 backgrounds, 2 corrupt: 100%|██████████| 2448/2448 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/jovyan/arquivos/Citologia/Datasets/Unificado/images_rescaled/yolo_format/val/images/2019_07_10__14_48__0045_b0s0c0x134053-1600y89742-1200m12460.tiff: 1 duplicate labels removed
val: WARNING ⚠️ /home/jovyan/arquivos/Citologia/Datasets/Unificado/images_rescaled/yolo_format/val/images/2019_07_10__14_48__0045_b0s0c0x157082-1600y90852-1200m12625.tiff: 1 duplicate labels removed
val: WARNING ⚠️ /home/jovyan/arquivos/Citologia/Datasets/Unificado/images_rescaled/yolo_format/val/images/2019_07_10__14_48__0045_b0s0c0x168684-1600y36895-1200m5183.tiff: 1 duplicate labels removed
val: WARNING ⚠️ /home/jovyan/arquivos/Citologia/Datasets/Unificado/images_rescaled/yolo_format/val/images/2019_07_10__14_48__0045_b0s0c0x78010-1600y13038-1200m1842.tiff: 1 duplicate labels removed
val: WARNING ⚠️ /home/jovyan/arquivos/Citologia/Datasets/Unificado/images_rescaled/yolo_format/val/images/2019_07_10__14_48__0045_b0s0c0x89480-1600y38971-1200m5426.tiff: ignoring corrupt image/label: non-nor


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 153/153 [10:54<00:00,  4.28s/it]


                   all       2446      16143      0.532      0.533      0.498      0.381      0.525      0.523      0.492      0.333
           pap_anormal        573       1534      0.463      0.825      0.641      0.498      0.468      0.817      0.641       0.46
          pap_saudavel        102        232      0.501      0.681      0.596      0.502      0.502      0.668      0.595       0.48
      pap_fora_de_foco        570       1673      0.405      0.408      0.373      0.252      0.414      0.384      0.377      0.232
         pap_sanguinea        281        638      0.451      0.622       0.49      0.369      0.453      0.598       0.49      0.329
        pap_em_divisao          4          4          1          0     0.0296     0.0294          1          0     0.0296     0.0267
      pap_nao_definida          9          9      0.618      0.111       0.12      0.106      0.632      0.111       0.12      0.107
         pap_reacional         48         85      0.417      0.219   

In [8]:
print(metrics_test.seg.map50)
print(metrics_test.seg.map75)
print(metrics_val.seg.map50)
print(metrics_val.seg.map75)

0.5002588215280105
0.40950478490625947
0.4918070646092144
0.406441173763082
